In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import Delaunay
from sklearn.metrics.cluster import fowlkes_mallows_score
import vg
import plotly.graph_objects as go
import ray
from pytransform3d.rotations import matrix_from_axis_angle
from scipy.spatial.transform import Rotation as Rot
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
import time
from numpy import transpose,array,hstack,arange,concatenate,eye,unique,dot,empty,append,ones,arccos
from numpy.linalg import norm
import warnings
warnings.filterwarnings('ignore')

def go_cluster(cluster):
    
    if(len(cluster.galaxies)==1):
        return [go.Scatter3d(
    x=cluster.galaxies[:,0],
    y=cluster.galaxies[:,1],
    z=cluster.galaxies[:,2],
    mode='markers',
    marker=dict(
        size=1,
        opacity=1,
        color='green'
    )
    )]
    
    return [go.Scatter3d(
    x=cluster.galaxies[:,0],
    y=cluster.galaxies[:,1],
    z=cluster.galaxies[:,2],
    mode='markers',
    marker=dict(
        size=3,
        opacity=1,
        color='green'

    )
    )]+[cluster.get_mesh()]+[go.Scatter3d(
    x=[cluster.centroid[0]],
    y=[cluster.centroid[1]],
    z=[cluster.centroid[2]],
    mode='markers',
    marker=dict(
        size=2,
        opacity=1,
        color='red'
    )
    )]

def get_clusters_plot(clusters,is_show=False,filename='3d_model'):
    
    data=[]
    for cluster in clusters:
        data+=go_cluster(cluster)
    
    fig = go.Figure(data=data)

# tight layout
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    if(is_show):
        fig.show()
    fig.write_html(filename+".html")


C:\Users\shan2\Anaconda3\lib\site-packages\ray\autoscaler\_private\cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [2]:
coor=pd.read_csv('ra_dec_z_data.csv')
coor_with_labels=pd.read_csv('coor_with_labels.csv')

In [3]:
tmp_coor_with_labels=coor_with_labels[['x','y','z','label']]
tmp_coor_with_labels=tmp_coor_with_labels[tmp_coor_with_labels.x.between(-50, 50)]
tmp_coor_with_labels=tmp_coor_with_labels[tmp_coor_with_labels.y.between(-50, 50)]
tmp_coor_with_labels=tmp_coor_with_labels[tmp_coor_with_labels.z.between(-50, 50)]

tmp_coor_with_labels.shape

(5260, 4)

In [4]:
tmp_coor=coor[['x','y','z']]
tmp_coor=tmp_coor[tmp_coor.x.between(30, 80)]
tmp_coor=tmp_coor[tmp_coor.y.between(-20, 20)]
tmp_coor=tmp_coor[tmp_coor.z.between(-20, 20)]
tmp_coor.shape

(1711, 3)

In [6]:

class Cluster():
    # non shifted shape, easy to expand
    non_rotated_cube=(transpose(array([[0, 0, 1, 1, 0, 0, 1, 1],[0, 1, 1, 0, 0, 1, 1, 0],[0, 0, 0, 0, 10, 10, 10, 10]]))-array([0.5,0.5,1.5]))*0.05
    rotated_cube=None# shifted coor of rotated cube
    centroid=None# coor of center
    galaxies=None

    is_complete=False
    times_grow=0
    def __init__(self, center,non_rotated_cube=None,galaxies=None):
        
        if(isinstance(non_rotated_cube, np.ndarray)):
            self.non_rotated_cube=non_rotated_cube
        if(not isinstance(galaxies, np.ndarray)):
            self.galaxies=array([center])
        else:
            self.galaxies=array(galaxies)
        
        self.centroid=center[:3]
        self.rotated_cube = self.rotate(self.centroid,self.non_rotated_cube)+self.centroid
        
        for gal in self.galaxies:
            if(Delaunay(self.rotated_cube).find_simplex(gal[:3]) < 0):
                raise ValueError
        
        
    def rotate(self, vector, points):    
        vector = vg.normalize(vector)
        axis = vg.perpendicular(vg.basis.z, vector)
        angle = vg.angle(vg.basis.z, vector, units='rad')
        
        a = hstack((axis, (angle,)))
        R = matrix_from_axis_angle(a)
        
        r = Rot.from_matrix(R)
        rotmat = r.apply(points)
        
        return rotmat

    def get_mesh(self):
        return go.Mesh3d(
        x=self.rotated_cube[:,0],
        y=self.rotated_cube[:,1],
        z=self.rotated_cube[:,2],
        opacity=0.5,
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],color='blue'
        )
    
    def grow(self,coef):
        if(self.times_grow==5):
            self.is_complete=True
            return
        self.times_grow+=1
        self.non_rotated_cube=self.non_rotated_cube*coef
        self.rotated_cube = self.rotate(self.centroid,self.non_rotated_cube)+self.centroid            
    
class Clusterer:
    
    unit_cube=(transpose(array([[0, 0, 1, 1, 0, 0, 1, 1],[0, 1, 1, 0, 0, 1, 1, 0],[0, 0, 0, 0, 1, 1, 1, 1]]))-0.5)
    
    def __init__(self):
        self.clusters=[]
        ray.init(log_to_driver=False)
    
    def merge(self):
        
        clusters_len=len(self.clusters)


        clusters_id = ray.put(self.clusters)

        graph=ray.get([self.parallel_connectivity_matrix.remote(self,clusters_id, subset) for subset in self.split_array(arange(len(self.clusters)),4)])

        graph = concatenate( graph, axis=0 )
        graph = graph+transpose(graph)
        graph=graph-eye(len(self.clusters))

        graph = csr_matrix(graph)
        n_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
        
        components=[]
        
        np_clusters=array(self.clusters)
        
        components=[np_clusters[labels==label]  for label in unique(labels) ]
 
        new_clusters=list(map(self.collide_clusters,components))

        self.clusters=new_clusters        
      

    def check_collision(self,a:Cluster, b: Cluster):      
        i=a.rotated_cube[1]-a.rotated_cube[0]
        j=a.rotated_cube[3]-a.rotated_cube[0]
        k=a.rotated_cube[4]-a.rotated_cube[0]
        for point in b.rotated_cube:
            u=point-a.rotated_cube[0]
            u_i,u_j,u_k=dot(u,i),dot(u,j),dot(u,k)
            if((0<= u_i)and(0<= u_j)and(0<= u_k)and
               (u_i <= dot(i,i))and(u_j <= dot(j,j))and(u_k <= dot(k,k))):
                return True
            
        i=b.rotated_cube[1]-b.rotated_cube[0]
        j=b.rotated_cube[3]-b.rotated_cube[0]
        k=b.rotated_cube[4]-b.rotated_cube[0]
        for point in a.rotated_cube:
            u=point-b.rotated_cube[0]
            u_i,u_j,u_k=dot(u,i),dot(u,j),dot(u,k)
            if((0<= u_i)and(0<= u_j)and(0<= u_k)and
               (u_i <= dot(i,i))and(u_j <= dot(j,j))and(u_k <= dot(k,k))):
                return True
        return False
 
    def collide_clusters(self, clusters:[Cluster]):
        
        if(len(clusters)==1):
            return clusters[0]
        galaxies=[cluster.galaxies for cluster in clusters]
        vertex_points=[cluster.rotated_cube for cluster in clusters]

        galaxies=concatenate( galaxies, axis=0 )
        vertex_points=concatenate( vertex_points, axis=0 )
        
        center = galaxies[:,:3].mean(axis=0)

        projections = center * dot(galaxies[:,:3], transpose([center])) / dot(center, center)
        
        distances_on_line=norm(projections-center,axis=1)
        vectors_from_line=galaxies[:,:3]-projections
    
        length = distances_on_line.max()
        width=norm(vectors_from_line,axis=1).max()
    
        cube=self.unit_cube.copy()*[width*2+0.05,width*2+0.05,length*2+0.05]
    
        return Cluster(center,cube,galaxies)
            
    def compress_cluster(self,cluster):

        galaxies=cluster.galaxies
        if(galaxies.ndim==1 ):
            galaxies=array([galaxies])
        vertex_points=cluster.rotated_cube
        
        center = galaxies[:,:3].mean(axis=0)

        projections = center * dot(galaxies[:,:3], transpose([center])) / dot(center, center)
        
        distances_on_line=norm(projections-center,axis=1)
        vectors_from_line=galaxies[:,:3]-projections
    
        length = distances_on_line.max()
        width=norm(vectors_from_line,axis=1).max()
    
        cube=self.unit_cube.copy()*[width*2+0.05,width*2+0.05,length*2+0.05]
    
        return Cluster(center,cube,galaxies)
    
    def step(self,grow_coef):
        
        is_done=True
        for cluster in self.clusters:
            if(not cluster.is_complete):
                is_done=False
                break
        if (is_done):
            return False
        
        for cluster in self.clusters:
            cluster.grow(grow_coef)
        self.merge()
        return True
      
    def split_array(self,a, n):
        k, m = divmod(len(a), n)
        return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))
    
    @ray.remote
    def parallel_connectivity_matrix(self,clusters ,rows):
        new_graph=empty((len(rows),len(clusters)))
        for i in range(len(rows)):
            for j in range(rows[i],len(clusters)):
                cluster_j=clusters[j]
                cluster_i=clusters[rows[i]]
                if(norm(cluster_i.centroid-cluster_j.centroid)>8):
                    continue 
                    
#                 if(norm(cluster_i.centroid-cluster_j.centroid)>min(3,3*max(cluster_i.non_rotated_cube[4,2],cluster_j.non_rotated_cube[4,2]))):
#                     continue
                if(arccos((dot(cluster_i.centroid, cluster_j.centroid) / (norm(cluster_i.centroid) * norm(cluster_j.centroid))))>0.01):
                    continue

                if(self.check_collision(cluster_i,cluster_j)):
                    new_graph[i,j]=1
        return new_graph
    
    def fit(self,data):
        coefs=1+1/arange(1,100)[15:]*5
        data_np=array(data)
        self.clusters=[Cluster(concatenate([data_np[i],[i]])) for i in range(len(data_np)) ]
        iter_num=1
        start = time.time()
        while(self.step(coefs[iter_num-1])):
            print('iter : ',iter_num,', n_clusters: ', len(self.clusters),', time: ',time.time()-start,' s')
            iter_num+=1
            start = time.time()
        
        galaxies=[]


        galaxies=[append(self.clusters[i].galaxies, ones((len(self.clusters[i].galaxies),1))*i, axis=1) for i in range(len(self.clusters))]
        self.clusters=[self.compress_cluster(cluster) for cluster in self.clusters]
        galaxies=concatenate(galaxies)
        galaxies = galaxies[galaxies[:,3].argsort()]  
        return galaxies[:,4]
        

In [7]:
%%time
clusterer=Clusterer()
labels=clusterer.fit(tmp_coor_with_labels[['x','y','z']])
print(fowlkes_mallows_score(tmp_coor_with_labels['label'],labels))
ray.shutdown()
get_clusters_plot(clusterer.clusters)

2021-05-09 01:53:44,265	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


iter :  1 , n_clusters:  5123 , time:  97.4392614364624  s
iter :  2 , n_clusters:  5059 , time:  96.07265400886536  s
iter :  3 , n_clusters:  4946 , time:  92.90786838531494  s
iter :  4 , n_clusters:  4776 , time:  80.96513295173645  s
iter :  5 , n_clusters:  4551 , time:  76.30889320373535  s
iter :  6 , n_clusters:  4499 , time:  66.70068454742432  s
iter :  7 , n_clusters:  4451 , time:  64.59830069541931  s
iter :  8 , n_clusters:  4385 , time:  63.62450957298279  s
iter :  9 , n_clusters:  4338 , time:  61.601282358169556  s
iter :  10 , n_clusters:  4302 , time:  60.32941436767578  s
iter :  11 , n_clusters:  4278 , time:  59.509357929229736  s
iter :  12 , n_clusters:  4259 , time:  58.130605697631836  s
iter :  13 , n_clusters:  4246 , time:  59.85214376449585  s
iter :  14 , n_clusters:  4237 , time:  63.33300590515137  s
iter :  15 , n_clusters:  4227 , time:  63.40834426879883  s
iter :  16 , n_clusters:  4225 , time:  62.80030274391174  s
iter :  17 , n_clusters:  4223 

In [ ]:
ray.shutdown()


Wall time: 2min 5s
Python 3.7.4
 13min 58s

In [11]:
import hdbscan
model=hdbscan.HDBSCAN().fit(tmp_coor_with_labels[['x','y','z']])
labels =model.labels_
c=1
for i in range(len(labels)):
    if(labels[i]==-1):
        labels[i]=400000+c
        c+=1
print(fowlkes_mallows_score(tmp_coor_with_labels['label'],labels))

0.054881264605726364


In [ ]:
def compress_cluster(galaxies):

    galaxies=galaxies
    if(galaxies.ndim==1 ):
        galaxies=array([galaxies])
    
    center = galaxies[:,:3].mean(axis=0)

    projections = center * dot(galaxies[:,:3], transpose([center])) / dot(center, center)
    
    distances_on_line=norm(projections-center,axis=1)
    vectors_from_line=galaxies[:,:3]-projections

    length = distances_on_line.max()
    width=norm(vectors_from_line,axis=1).max()

    cube=(transpose(array([[0, 0, 1, 1, 0, 0, 1, 1],[0, 1, 1, 0, 0, 1, 1, 0],[0, 0, 0, 0, 1, 1, 1, 1]]))-0.5)*[width*2+0.05,width*2+0.05,length*2+0.05]

    return Cluster(center,cube,galaxies)

In [ ]:
gals=[tmp_coor_with_labels[tmp_coor_with_labels['label']==label][['x','y','z']].to_numpy() for label in tmp_coor_with_labels['label'].unique()]
clus=list(map(compress_cluster,gals))
angle_list=[]
for cluster_i in clus:
    min_angle=1
    for cluster_j in clus:
        if(norm(cluster_i.centroid-cluster_j.centroid)>3):
            continue
        new_min=arccos((dot(cluster_i.centroid, cluster_j.centroid) / (norm(cluster_i.centroid) * norm(cluster_j.centroid))))
        if((new_min<min_angle) and (new_min!=0) ):
            min_angle=new_min
    if(min_angle<0.5):
        angle_list.append(min_angle)
print(min_angle)


In [ ]:
plt.hist(angle_list,bins=100)

In [9]:
tmp_coor_with_labels['label'].value_counts().value_counts()

1      2248
2       392
3       121
4        54
5        37
6        20
8        15
9        10
7         8
10        7
13        7
14        5
12        4
18        3
15        3
11        3
19        2
50        2
17        2
42        1
72        1
38        1
28        1
26        1
22        1
20        1
16        1
231       1
Name: label, dtype: int64

In [10]:
pd.Series(labels).value_counts().value_counts()

10      1
83      1
5167    1
dtype: int64